# Итераторы

Циклы `for` работают медленнее `while`. Одна из причин - вызов функции `iterate`:

```jl
for i in iter   # или  "for i = iter" или "for i ∈ iter"
    # body
end
```

внутренне работает так же как:

```jl
next = iterate(iter)
while next !== nothing
    (i, state) = next
    # body
    next = iterate(iter, state)
end
```

То же самое относится к comprehensions и generators.

Обратите внимание, что 'nothing' - это одноэлементное значение (единственное значение его типа `Nothing`), используемое соглашением, когда нет возвращаемого значения (немного похоже на` void` в C). Например

In [ ]:
typeof(print("hello"))

In [ ]:
A = ['a','b','c'];

In [ ]:
iterate(A)

In [ ]:
iterate(A, 2)

In [ ]:
iterate(A, 3)

In [ ]:
iterate(A, 4)

Итерация также используется при «деструктурировании» присваивания:

In [ ]:
x, y = A

In [ ]:
x

In [ ]:
y

Еще один пользователь этого 'протокола итерации' - это так называемый аргумент 'splatting':

In [ ]:
string(A)

In [ ]:
string('a','b','c')

In [ ]:
string(A...)

## Итерационные утилиты

`collect` предоставляет все элементы итератора в виде массива. Включения (`[... for ...]`) фактически эквивалентны вызову `collect` на генераторе.

In [ ]:
collect(pairs(A))

In [ ]:
collect(zip(100:102,A))

Некоторые другие фавориты для экспериментов. Они находятся во встроенном модуле `Iterators`:
- `enumerate`
- `rest`
- `take`
- `drop`
- `product`
- `flatten`
- `partition`

Некоторые итераторы бесконечны!
- `countfrom`
- `repeated`
- `cycle`

In [ ]:
I = zip(Iterators.cycle(0:1), Iterators.flatten([[2,3],[4,5]]))

In [ ]:
collect(I)

In [ ]:
collect(Iterators.product(I,A))

In [ ]:
string(I...)

## Определение итераторов

In [ ]:
struct SimpleRange
    lo::Int
    hi::Int
end

In [ ]:
Base.iterate(r::SimpleRange, state = r.lo) = state > r.hi ? nothing : (state, state+1)

In [ ]:
Base.length(r::SimpleRange) = r.hi-r.lo+1

In [ ]:
collect(SimpleRange(2,8))

## Особенности итератора

Для многих алгоритмов полезно заранее знать некоторые свойства итератора. Наиболее полезным является то, имеет ли итератор фиксированную, известную длину.

In [ ]:
Base.IteratorSize([1])

In [ ]:
Base.IteratorSize(Iterators.repeated(1))

In [ ]:
Base.IteratorSize(eachline(open("/dev/null")))

## Упражнение

Определите итератор, дающий первые N чисел Фибоначчи.

## Индексные итераторы

In [ ]:
A = rand(3,5)

In [ ]:
eachindex(A)

In [ ]:
keys(A)

In [ ]:
Av = view(A, [1,3], [1,2,5])

In [ ]:
A[[1,3],[1,2,5]]

In [ ]:
eachindex(Av)

### Пример: $3\times 3\times \dots \times3$ фильтр вагонов (из блога Тима Холи)

In [ ]:
function boxcar3(A::AbstractArray)
    out = similar(A)
    R = CartesianIndices(size(A))
    I1, Iend = first(R), last(R)
    for I in R
        n, s = 0, zero(eltype(out))
        for J in CartesianIndices(map(:, max(I1, I-I1).I, min(Iend, I+I1).I))
            s += A[J]
            n += 1
        end
        out[I] = s/n
    end
    out
end

In [ ]:
using Images

In [ ]:
A = rand(256,256);

In [ ]:
Gray.(A)

In [ ]:
Gray.(boxcar3(A))

In [ ]:
function sumalongdims!(B, A)
    # Предполагается, что B имеет размер 1 вдоль любого измерения, которое мы суммируем
    fill!(B, 0)
    Bmax = CartesianIndex(size(B))
    for I in CartesianIndices(size(A))
        B[min(Bmax,I)] += A[I]
    end
    B
end

In [ ]:
B = zeros(1, 256)

In [ ]:
sumalongdims!(B, A)

In [ ]:
reduce(+,A,dims=(1,))

`CartesianIndices` и другие 'N-d' итераторы имеют форму, которая распространяется через генераторы.

In [ ]:
[1 for i in CartesianIndices((2,3))]

In [ ]:
B = rand(5,5)

In [ ]:
view(B,CartesianIndices((2,3)))

## Упражнение: Жизнь CartesianIndex!

- Написать функцию `neighborhood(A::Array, I::CartesianIndex)` которая возвращает окрестность 3х3 вокруг определеного элемента
- Написать функцию `liferule(A, I)` которая реализует правило эволюции жизни клеточного автомата Конвея:
  - 2 Живых соседа $\rightarrow$ оставайся таким же
  - 3 Живых соседа $\rightarrow$ 1
  - в противном случае $\rightarrow$ 0
- Написать функцию `life(A)` которая отображает А на следующий шаг жизни, используя функции из предыдущих пунктов

Некоторые известные начальные условия:

In [ ]:
A = fill(0, 128,128);

In [ ]:
A[61:63,61:63] = [1 1 0
                  0 1 1
                  0 1 0]

In [ ]:
A = life(A)
# `repeat` can be used to get chunky pixels to make the output easier to see
Gray.(repeat(A,inner=(4,4)))